In [108]:
## Get the data from api call as json
import requests 

"""
  This function returns a json record returning all crypto information from a crpyto api request call 
  Input: None 
  Return: List(dict) json_record - a list of dictionary (not nested) containing data on crypto currencies
"""
def get_json_record():
  request_url = 'https://api.coinmarketcap.com/v1/ticker/?limit=0'
  response = requests.get(request_url)
  json_record= response.json()
  return json_record 

json_record = get_json_record()
print "Pulled data from api.coinmarketcap.com.."

Pulled data from api.coinmarketcap.com..


In [107]:
### Convert json to csv for loading into bigquery
import csv 

"""
  This function takes a json record, output filename, and a schema and produces a CSV file 
  Input:
    dict json_record - a list of dictionary (not nested) containing data on crypto currencies
    string filename - output filename
    list schema - schema in json_record to output to file 
  Output: 
  
"""
def gen_csv_file(json_record,filename,schema):
  outfile = open(filename,'w+')
  writer = csv.writer(outfile)
  


  writer.writerow(schema)
  for record in json_record:

    out_row = []

    for key in schema:
      
        ## The other option is to use a try, except and if the key does not exists in schema throw a error
        ## Depends on how enforcable the schema should be (if every col in the schema should exists in json_record dict) 
        if key in record: 
          out_row.append(record[key])
        else:
          out_row.append(None)

    writer.writerow(out_row)

schema = ['market_cap_usd', 'price_usd', 'last_updated', 'name', '24h_volume_usd', 'percent_change_7d', 'symbol', 'max_supply', 'rank', 'percent_change_1h', 'total_supply', 'price_btc', 'available_supply', 'percent_change_24h', 'id']
csv_filename = "crypto.csv"
import os 
if not os.path.exists(csv_filename):
  gen_csv_file(json_record,csv_filename,schema)
  print "Generated Output File: {}".format(csv_filename)
else:
  print "Error: File Already Exists.."
  ## If you wanted to redo this, you would need to remove file and rerun this funciton to regenerate the csv file 



File Already Exists..


In [109]:
import google.datalab.storage as storage

"""
  This function creates a bucket in gcs storage
  Input:
    string bucket_name: name of the bucket
  Output: 
    None
"""
def create_bucket(bucket_name):
  caserta_bucket = storage.Bucket("caserta-inteview")
  if not caserta_bucket.exists():
    caserta_bucket.create()
    print "Bucket {} created..".format(bucket_name)
  else:
    print "Error: Bucket {} already exists..".format(bucket_name)

"""
  This function loads a csv file into a specified gcs bucket
  Input:
    string bucket_name: name of the bucket
    file csv_file: the csv_file to load 
  Output: 
    string uri: the uri to the gcs path of the file 
"""
def load_file(bucket_name,csv_filename):
  
  file_path = None 
  csv_file = open(csv_filename,'r+')
  gcs_crypto_file = storage.Object(bucket_name,csv_filename)
  if gcs_crypto_file.exists():
    print "Error: crypto file is already loaded"
    return gcs_crypto_file.uri
  else:
    print "Loading crypto file"
    ## This can be wrapped in a try,except clause if further information or action is nessecary, or to prevent an application from stopping
    ## However in this case, I am going to let the error bubble up if it occurs 
    gcs_crypto_file.write_stream(csv_file.read(),'text/plain')
    print "Loaded File: {}".format(gcs_crypto_file.uri)
    
    return gcs_crypto_file.uri

bucket_name = "caserta-inteview"
create_bucket(bucket_name)
print 
crypto_file_path = load_file(bucket_name,csv_filename)

## I loaded the dataset using the UI, I can load it using the bq module but out of the sake of time I decided to skip doing this.

Error: Bucket caserta-inteview already exists..

Loading crypto file
Loaded File: gs://caserta-inteview/crypto.csv


In [99]:
## Predefined the table schema (however this can be inferred from the file)
table_schema = [
    {
      "type": "FLOAT",
      "name": "market_cap_usd",
      "mode": "NULLABLE"
    },
    {
      "type": "FLOAT",
      "name": "price_usd",
      "mode": "NULLABLE"
    },
    {
      "type": "INTEGER",
      "name": "last_updated",
      "mode": "NULLABLE"
    },
    {
      "type": "STRING",
      "name": "name",
      "mode": "NULLABLE"
    },
    {
      "type": "FLOAT",
      "name": "_24h_volume_usd",
      "mode": "NULLABLE"
    },
    {
      "type": "FLOAT",
      "name": "percent_change_7d",
      "mode": "NULLABLE"
    },
    {
      "type": "STRING",
      "name": "symbol",
      "mode": "NULLABLE"
    },
    {
      "type": "FLOAT",
      "name": "max_supply",
      "mode": "NULLABLE"
    },
    {
      "type": "INTEGER",
      "name": "rank",
      "mode": "NULLABLE"
    },
    {
      "type": "FLOAT",
      "name": "percent_change_1h",
      "mode": "NULLABLE"
    },
    {
      "type": "FLOAT",
      "name": "total_supply",
      "mode": "NULLABLE"
    },
    {
      "type": "FLOAT",
      "name": "price_btc",
      "mode": "NULLABLE"
    },
    {
      "type": "FLOAT",
      "name": "available_supply",
      "mode": "NULLABLE"
    },
    {
      "type": "FLOAT",
      "name": "percent_change_24h",
      "mode": "NULLABLE"
    },
    {
      "type": "STRING",
      "name": "id",
      "mode": "NULLABLE"
    }
  ]
bq_schema = bq.Schema(definition=table_schema)
print "Created Table Schema Object.."

Created Table Schema Object..


In [101]:
import google.datalab.bigquery as bq

"""
  This function create a big query dataset
  Input:
    string: name of the bq dataset
  Output: 
    None 
"""
def create_crypto_dataset(dataset_name):
  crypto_dataset = bq.Dataset(dataset_name)
  if not crypto_dataset.exists():
    crypto_dataset.create()
    print "Created BQ Dataset: {}".format(dataset_name)
  else:
    print "Error: BQ Dataset {} alread exists".format(dataset_name)

"""
  This function create a big query table
  Input:
    string table_name: name of the bq table
    schema object bq_schema: schema for the given table 
  Output: 
    None 
"""
def create_crypto_table(table_name,bq_schema):
  crypto_table = bq.Table(table_name)
  if not crypto_table.exists():
    crypto_table.create(bq_schema)
    print "Created Table: {}".format(table_name)
  else:
    print "Error: Table {} already exists".format(table_name)

"""
  This function loads a csv file from GCS into the specified fable 
  Input:
    string table_name: name of the bq table
    string file_path: gcs file path for the csv file 
  Output: 
    None 
"""
## The other option was the create the table and load in one function
## Could of used the load function and infer the schema 
def load_crypto_table(table_name,file_path):
  crypto_table = bq.Table(table_name)
  # Could make this a separate function or read from config, but for sake of simplicity incorporating it into the function
  options = bq.CSVOptions(skip_leading_rows=1)
  if crypto_table.length <= 0:
    load_job = crypto_table.load(crypto_file_path,source_format='csv',mode='append',csv_options=options)
    print "Loading data from {0} into table {1}..".format(file_path,table_name)
    return load_job 
  else:
    print "# Existing Rows in table: {}".format(crypto_table.length)
    print "Error:  Data already exists in table {}.. Data Load Skiped..".format(table_name)
    return None 
  

dataset_name = "Crypto"
crypto_dataset= bq.Dataset(dataset_name)
table_name = "{}.crypto_currencies".format(dataset_name)

## Note: the function are written to be idepotent, thus can be ran multiple times without reloading/recreating tables/data
create_crypto_dataset(dataset_name)
create_crypto_table(table_name,bq_schema)
load_job = load_crypto_table(table_name,crypto_file_path)

if load_job:
  # Wait for table to complete loading
  load_job.result()


Error: BQ Dataset Crypto alread exists
Error: Table Crypto.crypto_currencies_2 already exists
# Existing Rows in table: 1569
Error:  Data already exists in table Crypto.crypto_currencies_2.. Data Load Skiped..


In [76]:
%%bq query
-- The count of crypto with price in USD > 8k 
-- Its in the gutters :( 
SELECT  count(*) FROM Crypto.crypto_currencies  where price_usd > 8000



f0_
5


In [18]:
%%bq query
-- Total market cap of top 100 crypto in market cap 
select sum(market_cap_usd) from (
select * from Crypto.crypto_currencies order by market_cap_usd desc limit 100) a;

f0_
3.01048805156e+11


In [13]:
%%bq query
-- All cryptos with available supper < 5 mil
select name,available_supply from Crypto.crypto_currencies where available_supply < 5000000


name,available_supply
DigixDAO,2000000.0
Zcash,3700231.0
Byteball Bytes,645222.0
Maker,618228.0
Veritaseum,2036645.0
Mixin,411682.0
Elastos,4975104.0
ZCoin,4528741.0


In [15]:
%%bq query
-- Top 5 currencies with higest percent change is last week
select name,percent_change_7d from Crypto.crypto_currencies order by percent_change_7d desc limit 5

name,percent_change_7d
Golem,116.62
Mithril,107.45
Bytom,63.68
Wanchain,59.33
DigiByte,59.15


In [17]:
%%bq query
-- Crypto with X in symbol
select count(symbol) from Crypto.crypto_currencies where symbol like '%X%'

f0_
15
